# Sesión 15: Clasificador K-Nearest Neighbors (I)

## K-Nearest Neighbors

El modelo para kNN es todo el conjunto de datos de entrenamiento. Cuando se requiere una predicción para una instancia de datos invisible, el algoritmo kNN buscará en el conjunto de datos de entrenamiento las instancias más similares. El atributo de predicción de las instancias más similares se resume y devuelve como la predicción para la instancia invisible.

La medida de similitud depende del tipo de datos. Para datos de valor real, se puede usar la distancia euclidiana.

## Como trabaja k-Nearest Neighbors

El algoritmo kNN pertenece a la familia de algoritmos de aprendizaje competitivo y aprendizaje perezoso basados en instancias.

Los algoritmos basados en instancias son aquellos algoritmos que modelan el problema utilizando instancias de datos (o filas) para tomar decisiones predictivas. El algoritmo kNN es una forma extrema de métodos basados en instancias porque todas las observaciones de entrenamiento se retienen como parte del modelo.

Es un algoritmo de aprendizaje competitivo, porque internamente utiliza la competencia entre elementos del modelo (instancias de datos) para tomar una decisión predictiva. La medida objetiva de similitud entre instancias de datos hace que cada instancia de datos compita para "ganar" o sea más similar a una instancia de datos invisible y contribuya a una predicción.

El aprendizaje diferido se refiere al hecho de que el algoritmo no construye un modelo hasta el momento en que se requiere una predicción. Es vago porque solo funciona en el último segundo. Esto tiene la ventaja de incluir solo datos relevantes para los datos invisibles, denominado modelo localizado. Una desventaja es que puede ser costoso desde el punto de vista informático repetir las mismas o similares búsquedas en conjuntos de datos de entrenamiento más grandes.

Finalmente, kNN es poderoso porque no asume nada sobre los datos, aparte de que una medida de distancia puede calcularse de manera consistente entre dos instancias. Como tal, se llama no paramétrico o no lineal, ya que no asume una forma funcional.

## Ejemplo usando la data iris

In [ ]:
# read the iris data into a DataFrame
import pandas as pd
url = 'http://archive.ics.uci.edu/ml/machine-learning-databases/iris/iris.data'
col_names = ['sepal_length', 'sepal_width', 'petal_length', 'petal_width', 'species']
iris = pd.read_csv(url, header=None, names=col_names)

In [ ]:
iris.head()

In [ ]:
iris['species'].value_counts()

In [ ]:
# allow plots to appear in the notebook
%matplotlib inline
import matplotlib.pyplot as plt

# increase default figure and font sizes for easier viewing
plt.rcParams['figure.figsize'] = (6, 4)
plt.rcParams['font.size'] = 14

# create a custom colormap
from matplotlib.colors import ListedColormap
cmap_bold = ListedColormap(['#FF0000', '#00FF00', '#0000FF'])

In [ ]:
# map each iris species to a number
iris['species_num'] = iris.species.map({'Iris-setosa':0, 'Iris-versicolor':1, 'Iris-virginica':2})

In [ ]:
iris.head()

In [ ]:
# create a scatter plot of PETAL LENGTH versus PETAL WIDTH and color by SPECIES
iris.plot(kind='scatter', x='petal_length', y='petal_width', c='species_num', colormap=cmap_bold)

In [ ]:
# create a scatter plot of SEPAL LENGTH versus SEPAL WIDTH and color by SPECIES
iris.plot(kind='scatter', x='sepal_length', y='sepal_width', c='species_num', colormap=cmap_bold)

## Creando un clasificador KNN

### Estimando similaridad

Para hacer predicciones, necesitamos calcular la similitud entre dos instancias de datos dadas. Esto es necesario para que podamos localizar las k instancias de datos más similares en el conjunto de datos de entrenamiento para un miembro dado del conjunto de datos de prueba y, a su vez, hacer una predicción.

Dado que las cuatro medidas de flores son numéricas y tienen las mismas unidades, podemos usar directamente la medida de distancia euclidiana. Esto se define como la raíz cuadrada de la suma de las diferencias cuadráticas entre las dos matrices de números (léala varias veces y deje que se hunda).

Además, queremos controlar qué campos incluir en el cálculo de la distancia. Específicamente, solo queremos incluir los primeros 4 atributos. Un enfoque es limitar la distancia euclidiana a una longitud fija, ignorando la dimensión final.

Al unir todo esto, podemos definir la función `euclideanDistance` de la siguiente manera:

In [ ]:
import numpy as np
def euclideanDistance(instance1, instance2):
    distance = (instance1 - instance2) ** 2
    # Check if either instance1 or instance2 is a matrix
    if distance.shape[0] == distance.size:
        return distance.sum() ** 0.5
    else:
        return distance.sum(axis=1) ** 0.5

In [ ]:
data1 = np.array([2, 2, 1])
data2 = np.array([4, 4, 1])

distance = data1 - data2
distance

In [ ]:
data1 = np.array([2, 2])
data2 = np.array([4, 4])
distance = euclideanDistance(data1, data2)
print('Distance: ' + repr(distance))

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
plt.scatter(data1[0], data1[1])
plt.scatter(data2[0], data2[1])
plt.plot([data1[0], data2[0]], [data1[1], data2[1]], '--r')

### Encontrando Vecinos


Ahora que tenemos una medida de similitud, podemos usarla para recopilar las k instancias más similares para una instancia invisible.

Este es un proceso sencillo de calcular la distancia para todas las instancias y seleccionar un subconjunto con los valores de distancia más pequeños.

A continuación se muestra la función `getNeighbours` que devuelve k vecinos más similares del conjunto de entrenamiento para una instancia de prueba dada (utilizando la función` euclideanDistance` ya definida)

In [ ]:
trainSet = np.array([[2, 2], [4, 4], [7, 7], [4, 1], [3, 4], [5, 2]])
testInstance = np.array([5, 5])

In [ ]:
dist = euclideanDistance(trainSet, testInstance)
dist

Vemos cuáles son los puntos más cercanos:

In [ ]:
dist.argsort()[:2]

In [ ]:
def getNeighbors(trainSet, testInstance, k):
    dist = euclideanDistance(trainSet, testInstance)
    neighbors = dist.argsort()[:k]
    return neighbors

In [ ]:
k=2
neighbors = getNeighbors(trainSet, testInstance, k)
print(neighbors)

In [ ]:
plt.scatter(trainSet[:, 0], trainSet[:, 1], s=50)
plt.scatter(testInstance[0], testInstance[1], c='green', s=100)
plt.plot([testInstance[0], trainSet[1, 0]], [testInstance[1], trainSet[1, 1]], '--r')

In [ ]:
testInstance = np.array([3.4, 3])
k = 3
neighbors = getNeighbors(trainSet, testInstance, k)
print(neighbors)

In [ ]:
plt.scatter(trainSet[:, 0], trainSet[:, 1], s=50)
plt.scatter(testInstance[0], testInstance[1], c='green', s=100)
for neighbor in neighbors:
    plt.plot([testInstance[0], trainSet[neighbor, 0]], [testInstance[1], trainSet[neighbor, 1]], '--r')

### Respuesta

Una vez que hemos localizado los vecinos más similares para una instancia de prueba, la siguiente tarea es diseñar una respuesta pronosticada basada en esos vecinos.

Podemos hacer esto permitiendo que cada vecino vote por su atributo de clase y tome el voto mayoritario como la predicción.

Primero definamos la etiqueta de cada instancia.

In [ ]:
trainSet_y = np.array([0, 0, 1, 0, 1, 1])

In [ ]:
plt.scatter(trainSet[trainSet_y==0, 0], trainSet[trainSet_y==0, 1], s=50)
plt.scatter(trainSet[trainSet_y==1, 0], trainSet[trainSet_y==1, 1], c='y', s=50)

A continuación se proporciona una función para obtener la respuesta votada por la mayoría de varios vecinos. Se supone que la clase es el último atributo para cada vecino.

In [ ]:
plt.scatter(trainSet[trainSet_y==0, 0], trainSet[trainSet_y==0, 1], s=50)
plt.scatter(trainSet[trainSet_y==1, 0], trainSet[trainSet_y==1, 1], c='y', s=50)
plt.scatter(testInstance[0], testInstance[1], c='green', s=100)
for neighbor in neighbors:
    plt.plot([testInstance[0], trainSet[neighbor, 0]], [testInstance[1], trainSet[neighbor, 1]], '--r')

In [ ]:
trainSet_y[neighbors]

In [ ]:
from scipy.stats import itemfreq
freq = itemfreq(trainSet_y[neighbors])
freq

In [ ]:
freq2 = pd.Series(trainSet_y[neighbors])
freq2.value_counts().index[0]

In [ ]:
freq[:, 0][freq[:, 1].argmax()]

In [ ]:
freq[:, 1] / freq[:, 1].sum()

In [ ]:
np.vstack((freq[:, 0], freq[:, 1] / freq[:, 1].sum())).T

In [ ]:
def getResponse(trainSet_y, neighbors):
    votes = trainSet_y[neighbors]
    freq = itemfreq(votes)
    return freq[:, 0][freq[:, 1].argmax()], np.vstack((freq[:, 0], freq[:, 1] / freq[:, 1].sum())).T

In [ ]:
response = getResponse(trainSet_y, neighbors)
print(response)

### Classifier

Lets put everything together

In [ ]:
def knn_classifier_one(trainSet, trainSet_y, testInstance, k):
    neighbors = getNeighbors(trainSet, testInstance, k)
    pred_y, pred_prob = getResponse(trainSet_y, neighbors)
    return pred_y, pred_prob, neighbors

In [ ]:
testInstance = np.array([4.2, 4.1])
plt.scatter(trainSet[trainSet_y==0, 0], trainSet[trainSet_y==0, 1], s=50)
plt.scatter(trainSet[trainSet_y==1, 0], trainSet[trainSet_y==1, 1], c='y', s=50)
plt.scatter(testInstance[0], testInstance[1], c='green', s=100)

In [ ]:
for k in range(1, 6):
    print('k = ', k)
    pred_y, pred_prob, neighbors = knn_classifier_one(trainSet, trainSet_y, testInstance, k)
    print('pred_y = ', pred_y)
    print('pred_prob = ', pred_prob)
    plt.scatter(trainSet[trainSet_y==0, 0], trainSet[trainSet_y==0, 1], s=50)
    plt.scatter(trainSet[trainSet_y==1, 0], trainSet[trainSet_y==1, 1], c='y', s=50)
    plt.scatter(testInstance[0], testInstance[1], c='green', s=100)
    for neighbor in neighbors:
        plt.plot([testInstance[0], trainSet[neighbor, 0]], [testInstance[1], trainSet[neighbor, 1]], '--r')
    plt.show()

### Permitir más de una instancia

In [ ]:
testInstances = np.array([[4.2, 4.1], [1, 3], [6, 6]])
plt.scatter(trainSet[trainSet_y==0, 0], trainSet[trainSet_y==0, 1], s=50)
plt.scatter(trainSet[trainSet_y==1, 0], trainSet[trainSet_y==1, 1], c='y', s=50)
plt.scatter(testInstances[:,0], testInstances[:,1], c='green', s=100)

In [ ]:
def knn_classifier(trainSet, trainSet_y, testInstances, k):
    n_samples_test = testInstances.shape[0]
    pred_y = np.zeros(n_samples_test)
    y_unique = np.unique(trainSet_y)
    pred_prob = np.zeros((n_samples_test, y_unique.shape[0]))
    for i in range(n_samples_test):
        neighbors = getNeighbors(trainSet, testInstances[i], k)
        pred_y_, pred_prob_ = getResponse(trainSet_y, neighbors)
        pred_y[i] = pred_y_
        
        # pred_y may not include all values of y
        for j in range(y_unique.shape[0]):
            pred_prob[i, j] =  pred_prob_[pred_prob_[:,0] == y_unique[j], 1].sum()
            
    return pred_y, pred_prob

In [ ]:
k = 3
knn_classifier(trainSet, trainSet_y, testInstances, k)

## Aplicando a la data Iris

In [ ]:
y = iris.species_num
X = iris[['sepal_length', 'sepal_width', 'petal_length', 'petal_width']]

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X.values, y.values, random_state=123)

In [ ]:
y_pred, y_pred_prob = knn_classifier(X_train, y_train, X_test, k=5)

In [ ]:
y_pred_prob[:5]

In [ ]:
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test, y_pred)

## Using Sklearn

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

In [ ]:
knn = KNeighborsClassifier(n_neighbors=5)

In [ ]:
knn.fit(X_train, y_train)

In [ ]:
y_pred = knn.predict(X_test)
y_pred_prob = knn.predict_proba(X_test)

In [ ]:
y_pred_prob[:5]

In [ ]:
confusion_matrix(y_test, y_pred)

Referencia: https://www.coursera.org/learn/python-machine-learning